### Создание и нормализация базы данных.

#### Задание: ####

1) Продумать структуру базы данных и отрисовать в редакторе.
2) Нормализовать базу данных (1НФ — 3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась.
3) Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет (использовать команду CREATE TABLE).
4) Загрузить данные в таблицы в соответствии с созданной структурой (использовать команду INSERT INTO или загрузить файлы, используя возможности инструмента DBeaver; в случае загрузки файлами приложить скрины, что данные действительно были залиты).

#### 1. Продумать структуру базы данных и отрисовать в редакторе. ####

Исходные данные представляли собой две таблицы с плохо структурированной информацией.

![title](Схема_0.png)

После анализа данных я разработал схему, к которой необходимо привести исходные данные.

![title](Схема_1.png)

#### 2. Нормализовать базу данных (1НФ — 3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась. ####

Разработанная мной в предыдущем пункте схема нормализована до третьей нормальной формы. Для достижения этого я последовательно провёл анализ, сначала приведя её к первой нормальной форме, затем ко второй, и, наконец, к третьей.

##### Шаг 1. Приведение к первой нормальной форме (1NF). 

Требования 1NF:
- Каждая ячейка содержит только одно значение (данные атомарны).
- Наличие первичного ключа.
- Отсутствие повторяющихся групп данных.

Анализ:
- Все поля содержат атомарные данные.
- Обе таблицы уже имеют первичный ключ.
- Дублирующиеся данные отсутствуют.

Вывод: Обе таблицы уже были в 1NF.

##### Шаг 2. Приведение ко второй нормальной форме (2NF). 

Требования 2NF:
- Таблица должна быть в 1NF.
- Все неключевые атрибуты должны зависеть от всего первичного ключа, а не от его части.

Анализ:
- В *transaction* данные о товарах (*brand*, *product_line*, *product_class*, *product_size*, *standard_cost*) должны зависеть от продукта, а не от всей транзакции.
- В *customer* данные о работе (*job_title*, *job_industry_category*) должны зависеть от профессии, а не от клиента.
- В *customer* адрес должен зависеть от местоположения, а не от клиента.

Результат:
- Вынесли данные о товарах в таблицу *products*. Но так как у разных продуктов может быть один *product_id*, то требуется создать *product_new_id* для обеспечения уникальности первичного ключа в новой таблице.(*product_new_id* → *product_id*, *brand*, *product_line*, *product_class*, *product_size*, *standard_cost*).
- Вынесли данные о профессиях в таблицу job_categories (job_id → job_title, job_industry_category).
-  Вынесли данные об адресах в таблицу addresses (address_id → address, postcode, state, country, property_valuation).

Вывод: Все неключевые атрибуты зависят от всего первичного ключа, а не от его части.

##### Шаг 3. Приведение к третьей нормальной форме (3NF).

Требования 3NF:
- Таблица должна быть в 2NF.
- Каждый неключевой атрибут должен зависеть ТОЛЬКО от первичного ключа (а не от других неключевых атрибутов).

Анализ:
- Каждый неключевой атрибут зависит только от первичного ключа.

Вывод: Все транзитивные зависимости устранены.

#### 3. Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет (использовать команду CREATE TABLE).

Реализованная схема в DBeaver выглядит так:

![title](Схема_2.png)

**customers** (клиенты):
- customer_id PK (Уникальный id клиента) - INT NOT NULL (id это целое число, не может отсутствовать поскольку тогда не будет клиента)
- first_name (Имя) - VARCHAR(255) NOT NULL (имя это строка, не может отсутствовать у клиента, раз есть клиент, то у него должно быть имя)
- last_name (Фамилия) - VARCHAR(255) (фамилия это строка, может отсутствовать, поскольку не у всех в мире есть фамилия)
- gender (пол) - VARCHAR(10) NOT NULL (пол это строка, не может отсутствовать так как у каждого человека он есть и он важен)
- DOB (дата рождения) - DATE (строка с датой в формате *yyyy-mm-dd*, у каждого человека есть, но может отсутствовать так как не все его вносят в профель, а спрашивать возраст девушки вообще не приличчно)
- wealth_segment (сегмент благосостояния) - VARCHAR(50) (строка с фразой обозначающей сегмент благосостояния, может отсутствовать поскольку не всегда это можно оценить, да и не каждый захочет это указывать)
- deceased_indicator (флаг, жив клиент или мертв) - CHAR(1) NOT NULL (один символ, N - жив, Y - мертв, не может пустовать поскольку каждый клиент либо жив, либо мертв, это важная информация)
- own_car (владеет ли машиной) - VARCHAR(3) (слово YES или NO, не все хотят говорить есть ли у них машина, да и не всегда это важно, поэтому может пустовать)
- address_id FK (Уникальный id адреса) - INT (id это целое число, может отсутствовать поскольку не у всех есть адрес, да и не все его указывают)
- job_id FK (Уникальный id должности) - INT (id это целое число, может отсутствовать поскольку не у всех есть раота, да и не все указывают свою должность, даже если она есть)

**adresses** (адреса):
- address_id PK (Уникальный id адреса) - INT NOT NULL (id это целое число, не может отсутствовать поскольку тогда не будет и адреса)
- address (адрес) - VARCHAR(255) NOT NULL (адрес, в данном случае улица это строка, не может отсутствовать поскольку если указывается адрес, то нужно указывать его полностью)
- postcode (почтовый индекс) - VARCHAR(10) NOT NULL (строка из чисел, так как индекс может теоретически начинаться с 0, то берется не INT, а VARCHAR, не может отсутствовать, потому что у каждого адреса он есть, обычно вставляется автоматически)
- state (штат) - VARCHAR(50) NOT NULL (штат это строка, не может отсутствовать, поскольку одна и таже улица может быть в разных штатах, без него была бы путаница)
- country (страна) - VARCHAR(50) NOT NULL (страна это строка, не может отсутствовать, поскольку один и тот же адрес может быть в разных странах, без нее была бы путаница)
- property_valuation (оценочная стоимость недвижимости) INT (число, может отсутствовать так как не все знают эту стоимость, а даже те кто знают, не всегда станут ее где то указывать)


**job_categories** (профессия):
- job_id PK (Уникальный id клиента) - INT NOT NULL (id это целое число, не может отсутствовать поскольку тогда не будет указана профессия)
- job_title (должность) - VARCHAR(255) (должность это строка, не всега указывается, часто указывается только отрасль работы)
- job_industry_category (отрасль работы) - VARCHAR(100) NOT NULL (отрасль работы это строка, не может отсутствовать потому что просто указывая должность может быть непонятно что за профессия)


**product** (продукт):
- product_new_id PK (Уникальный id продукта) - INT NOT NULL (id это целое число, не может отсутствовать поскольку тогда не будет продукта)
- product_id (не уникальный id, зависит от других характиристик) - INT NOT NULL (id это целое число, не может отсутствовать поскольку тогда не будет продукта)
- brand (бренд) - VARCHAR(100) (бренд это строчка, может отсутствовать, так как может быть просто не известен)
- product_line (линейка товаров) - VARCHAR(100) (линейка товаров это строчка, может отсутствовать, так как может просто не иметь отдельной линейки товаров)
- product_class (класс товара) - VARCHAR(50) (класс товара это строчка, может отсутствовать, так как может просто не иметь отдельного класса товаров)
- product_size (размер товара) - VARCHAR(50) (размер товара это строчка, может отсутствовать, так как часто просто не указывается)
- standard_cost (себестоимость товара) - DECIMAL(10,2) (себестоимость товара это число с дробной частью, может отсутствовать поскольку не всегда известна тем кто продает)

**transactions** (транзакции):
- transaction_id PK (Уникальный id транзакции) - INT NOT NULL (id это целое число, не может отсутствовать поскольку тогда не будет и транзакции)
- customer_id FK (id клиента, который совершает транзакцию) - INT NOT NULL (id это целое число, не может отсутствовать поскольку у транзакции должен быть тот, кто ее совершает)
- product_new_id FK (id продукта, который покупает клиент) - INT (id это целое число, может отсутствовать поскольку в транзакции может не быть отмечено что покупается)
- transaction_date (дата транзакции) - DATE NOT NULL (дата в формате *yyyy-mm-dd*, не может отсутствовать поскольку у каждой транзакции должна быть дата)
- online_order (признак того была ли проведена транзакция онлайн) - BOOL (булевая переменная, TRUE - онлайн, False - офлайн, может отсутствовать потому как не очень важный параметр, иногда оставляют пустым)
- order_status (статус заказа) - VARCHAR(50) NOT NULL (строчка о состоянии заказа, не может быть пустой, поскольку это важно, прошел заказ или нет)
- list_price (цена продажи) - DECIMAL(10,2) NOT NULL (цена продажи это число с дробной частью, не может отсутствовать потому что это главный пункт в транзакции, за какую суммы бул продан товар)

SQL скрипт для создания данной схемы выглядит так:

~~~~sql
CREATE TABLE customers (
  customer_id INT NOT NULL PRIMARY KEY,
  first_name VARCHAR(255) NOT NULL,
  last_name VARCHAR(255),
  gender VARCHAR(10) NOT NULL,
  DOB DATE,
  wealth_segment VARCHAR(50),
  deceased_indicator CHAR(1) NOT NULL,
  owns_car 	VARCHAR(3),
  address_id INT,
  job_id INT
);

CREATE TABLE addresses (
  address_id INT NOT NULL PRIMARY KEY,
  address VARCHAR(255) NOT NULL,
  postcode VARCHAR(10) NOT NULL,
  state VARCHAR(50) NOT NULL,
  country VARCHAR(50) NOT NULL,
  property_valuation INT
);

CREATE TABLE job_categories (
  job_id INT NOT NULL PRIMARY KEY,
  job_title VARCHAR(255),
  job_industry_category VARCHAR(100) NOT NULL
);

CREATE TABLE products (
  product_new_id INT NOT NULL PRIMARY KEY,
  product_id INT NOT NULL,
  brand VARCHAR(100),
  product_line VARCHAR(100),
  product_class VARCHAR(50),
  product_size VARCHAR(50),
  standard_cost DECIMAL(10,2)
);

CREATE TABLE transactions (
  transaction_id INT NOT NULL PRIMARY KEY,
  customer_id INT NOT NULL,
  product_new_id INT,
  transaction_date DATE NOT NULL,
  online_order BOOL, 
  order_status VARCHAR(50) NOT NULL,
  list_price DECIMAL(10,2) NOT NULL
);

ALTER TABLE customers ADD CONSTRAINT address_id_fk FOREIGN KEY (address_id) REFERENCES addresses (address_id);
ALTER TABLE customers ADD CONSTRAINT job_id_fk FOREIGN KEY (job_id) REFERENCES job_categories (job_id);
ALTER TABLE transactions ADD CONSTRAINT customer_id_fk FOREIGN KEY (customer_id) REFERENCES customers (customer_id);
ALTER TABLE transactions ADD CONSTRAINT product_new_id_fk FOREIGN KEY (product_new_id) REFERENCES products (product_new_id)
~~~~



#### 4. Загрузить данные в таблицы в соответствии с созданной структурой (использовать команду INSERT INTO или загрузить файлы, используя возможности инструмента DBeaver; в случае загрузки файлами приложить скрины, что данные действительно были залиты) ####


Данный файл я загружал с помощью возможностей встроенных в DBeaver.

![title](customers.png)

![title](addresses.png)

![title](job_categories.png)

![title](products.png)

![title](transactions.png)

P.S. Была проблема при заливке данных, клиента с *customer_id = 5034* не существует, хотя в транзакциях он числится. Для решения этой проблемы в *customers* был добавлен данный клиент.